In [48]:
from copy import copy
from dataclasses import dataclass
from typing import Any, List


def evaluate(a):
    try:
        return copy(a.eval())
    except AttributeError:
        return copy(a)


def validate(a):
    if not (a in [0, 1] or isinstance(a, (str, And, Or, Xor, Not))):
        raise ValueError(
            "Value must be 0, 1, or an instance of str, And, Or, Xor, or Not."
        )
    else:
        return copy(a)


@dataclass
class And:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, _) | (_, 0):
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 0
            case (1, a) | (a, 1):
                return a
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}&{self.b})"


@dataclass
class Or:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (1, _) | (_, 1):
                return 1
            case (0, a) | (a, 0):
                return a
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}|{self.b})"


@dataclass
class Xor:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, 1) | (1, 0):
                return 1
            case (0, 0) | (1, 1):
                return 0
            case (0, a) | (a, 0):
                return a
            case (1, a) | (a, 1):
                return Not(a)
            case (a, b) if a == b:
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == b:
                return c
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == c:
                return b
            case (Xor(a, b), c) | (a, Xor(b, c)) if b == c:
                return a
            case _:
                return copy(self)

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}^{self.b})"


@dataclass
class Not:
    a: Any

    def __init__(self, a):
        self.a = validate(a)

    def eval(self):
        match evaluate(self.a):
            case 0:
                return 1
            case 1:
                return 0
            case Not(a):
                return a
            case _:
                return copy(self)

    def __str__(self):
        r = self.eval()
        if r in [0, 1]:
            return f"{r}"
        return f"~{self.a}"

In [49]:
# Example usage:
assert And("a", 0).eval() == 0
assert And("a", 1).eval() == "a"
assert And("a", Not("a")).eval() == 0
assert Or("a", 1).eval() == 1
assert Or("a", 0).eval() == "a"
assert Or("a", Not("a")).eval() == 1
assert Xor("a", 1).eval() == Not("a")
assert Xor("a", 0).eval() == "a"
assert Xor("a", Not("a")).eval() == 1
assert Xor("a", "a").eval() == 0
assert Xor("a", Xor("a", "b")).eval() == "b"
assert Not(Not("a")).eval() == "a"

In [50]:
@dataclass
class BitArray:
    bits: List

    def from_int(n, length):
        """Create a BitArray from an integer"""
        return BitArray([int(i) for i in bin(n)[2:].zfill(length)])

    def to_int(self):
        """Converts the BitArray to an integer. If the BitArray contains bits with unknown values, returns None."""
        if not all(isinstance(evaluate(bit), int) for bit in self.bits):
            return None
        return int("".join(str(bit) for bit in self.bits), 2)

    def len(self):
        """Returns the length of the BitArray"""
        return len(self.bits)

    def __and__(self, other):
        """Bitwise AND"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([And(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __or__(self, other):
        """Bitwise OR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Or(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __xor__(self, other):
        """Bitwise XOR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Xor(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __invert__(self):
        """Bitwise NOT"""
        return BitArray([Not(a) for a in self.bits])

    def __iand__(self, other):
        """In-place bitwise AND"""
        return self & other

    def __ior__(self, other):
        """In-place bitwise OR"""
        return self | other

    def __ixor__(self, other):
        """In-place bitwise XOR"""
        return self ^ other

    def __lshift__(self, n):
        """Shift left by n bits"""
        shifted = self.bits[n:] + [0] * n
        return BitArray(shifted)

    def __rshift__(self, n):
        """Shift right by n bits"""
        shifted = [0] * n + self.bits[:-n]
        return BitArray(shifted)

    def __str__(self):
        return "_".join(str(bit) for bit in self.bits)

    def reverse_bits(self):
        """Reverse the order of bits"""
        return BitArray(self.bits[::-1])

    def rotate_left(self, n):
        """Rotate left by n bits"""
        rotated = self.bits[n:] + self.bits[:n]
        return BitArray(rotated)

    def rotate_right(self, n):
        """Rotate right by n bits"""
        rotated = self.bits[-n:] + self.bits[:-n]
        return BitArray(rotated)

In [51]:
# Example usage:
a = BitArray(["a", "b", "c", "d"])
b = BitArray.from_int(0b1100, 4)

assert f"{a}" == "a_b_c_d"
assert f"{b}" == "1_1_0_0"
assert b.to_int() == 0b1100
assert f"{a & b}" == "a_b_0_0"
assert f"{(a | b)}" == "1_1_c_d"
assert f"{(a ^ b)}" == "~a_~b_c_d"
assert f"{(~a)}" == "~a_~b_~c_~d"
assert f"{(a << 2)}" == "c_d_0_0"
assert f"{(a >> 2)}" == "0_0_a_b"
assert f"{a.reverse_bits()}" == "d_c_b_a"
assert f"{a.rotate_left(1)}" == "b_c_d_a"
assert f"{a.rotate_right(1)}" == "d_a_b_c"

In [52]:
@dataclass
class U64:
    x: int

    def __init__(self, x):
        self.x = x & (2**64 - 1)

    def __and__(self, other):
        return U64(self.x & other.x)

    def __or__(self, other):
        return U64(self.x | other.x)

    def __xor__(self, other):
        return U64(self.x ^ other.x)

    def __invert__(self):
        return U64(~self.x & (2**64 - 1))

    def __lshift__(self, n):
        return U64(self.x << n)

    def __rshift__(self, n):
        return U64(self.x >> n)

    def __str__(self):
        return f"{self.x:#0{18}x}"

    def __int__(self):
        return self.x

    def rand():
        from random import randint

        return U64(randint(0, 2**64 - 1))

    def rotate_left(self, n):
        return U64((self.x << n) | (self.x >> (64 - n)))

    def rotate_right(self, n):
        return U64((self.x >> n) | (self.x << (64 - n)))


# Randomized tests for BitArray with 64 bits
count = 100
for i in range(count):
    a = U64.rand()
    b = U64.rand()

    arr_a = BitArray.from_int(int(a), 64)
    arr_b = BitArray.from_int(int(b), 64)

    assert int(a & b) == (arr_a & arr_b).to_int()
    assert int(a | b) == (arr_a | arr_b).to_int()
    assert int(a ^ b) == (arr_a ^ arr_b).to_int()
    assert int(~a) == (~arr_a).to_int()
    assert int(a << 2) == (arr_a << 2).to_int()
    assert int(a >> 2) == (arr_a >> 2).to_int()
    assert int(a.rotate_left(2)) == (arr_a.rotate_left(2)).to_int()
    assert int(a.rotate_right(2)) == (arr_a.rotate_right(2)).to_int()
    print(f"{i+1} of {count} tests passed.", end="\r")

In [53]:
def compress(x, mask):
    """Moves the masked bits in `x` to the right and sets the rest to 0."""
    from math import log2

    steps = int(log2(x.len()))
    assert 2**steps == x.len(), "BitArray length must be a power of 2."

    x &= mask
    mk = ~mask << 1
    for i in range(steps):
        mp = mk ^ (mk << 1)
        for j in range(steps - 1):
            power = 1 << (2**j)
            mp = mp ^ (mp << power)
        mv = mp & mask
        mask = (mask ^ mv) | (mv >> (1 << i))
        t = x & mv
        x = x ^ t | (t >> (1 << i))
        mk &= ~mp
    return x


def delta_swap(x, mask, shift):
    """Moves the masked bits to the left by `shift` positions. For this function to work properly,
    the mask and the shifted mask should not overlap, ie. `mask & (mask << shift) == 0` and no bits
    should be shifted out of the 64-bit integer, ie. `((mask << shift) >> shift) == mask`
    """
    t = ((x >> shift) ^ x) & mask
    return (x ^ t) ^ (t << shift)


def exchange(x, y, mask):
    """Exchanges the masked bits between `x` and `y`."""
    x = x ^ y
    y = y ^ (x & mask)
    x = x ^ y
    return x, y


x = BitArray(["a", "b", "c", "d", "e", "f", "g", "h"])
y = BitArray(["1", "2", "3", "4", "5", "6", "7", "8"])

mask = BitArray([0, 0, 0, 1, 0, 0, 0, 1])

swapped = delta_swap(x, mask, 3)
assert f"{swapped}" == "d_b_c_a_h_f_g_e"

x, y = exchange(x, y, mask)
assert f"{x}" == "a_b_c_4_e_f_g_8"
assert f"{y}" == "1_2_3_d_5_6_7_h"

assert f"{compress(x, mask)}" == "0_0_0_0_0_0_4_8"
assert f"{compress(y, mask)}" == "0_0_0_0_0_0_d_h"

In [54]:
# Transposing a 8x8 bit matrix stored in a 64-bit integer:

# Input:    Mask 0:   Step 0:   Mask 1:   Step 1:  Mask 2:   Result:
# .$ZYXWVU  00000000  .TZRXPVN  00000000  .TLDXPHz 00000000  .TLDvnf7
# TSRQPONM  10101010  $SYQWOUM  00000000  $SKCWOGy 00000000  $SKCume6
# LKJIHGFE  00000000  LDJBHzFx  11001100  ZRJBVNFx 00000000  ZRJBtld5
# DCBAzyxw  10101010  KCIAGyEw  11001100  YQIAUMEw 00000000  YQIAskc4
# vutsrqpo  00000000  vntlrjph  00000000  vnf7rjb3 11110000  XPHzrjb3
# nmlkjihg  10101010  umskqiog  00000000  ume6qia2 11110000  WOGyqia2
# fedcba98  00000000  f7d5b391  11001100  tld5ph91 11110000  VNFxph91
# 76543210  10101010  e6c4a280  11001100  skc4og80 11110000  UMEwog80

# fmt: off
bits = [
    '.','$','Z','Y','X','W','V','U',
    'T','S','R','Q','P','O','N','M',
    'L','K','J','I','H','G','F','E',
    'D','C','B','A','z','y','x','w',
    'v','u','t','s','r','q','p','o',
    'n','m','l','k','j','i','h','g',
    'f','e','d','c','b','a','9','8',
    '7','6','5','4','3','2','1','0',
]
# fmt: on

transpose = [bits[8 * i + j] for j in range(8) for i in range(8)]
matrix = BitArray(bits)

m0 = BitArray.from_int(0x00AA00AA00AA00AA, 64)
m1 = BitArray.from_int(0x0000CCCC0000CCCC, 64)
m2 = BitArray.from_int(0x00000000F0F0F0F0, 64)

assert matrix.len() == 64
assert m0.len() == 64
assert m1.len() == 64
assert m2.len() == 64

matrix = delta_swap(matrix, m0, 7)
matrix = delta_swap(matrix, m1, 14)
matrix = delta_swap(matrix, m2, 28)

assert matrix == BitArray(transpose)

In [60]:
# A 16x16 bit matrix stored in four 64-bit integers.

# We want to get from this:
# fmt: off
a = ['a00', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07', 'a08', 'a09', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15',
     'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'a31',
     'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47',
     'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55', 'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63']

b = ['b00', 'b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b09', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15',
     'b16', 'b17', 'b18', 'b19', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29', 'b30', 'b31',
     'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39', 'b40', 'b41', 'b42', 'b43', 'b44', 'b45', 'b46', 'b47',
     'b48', 'b49', 'b50', 'b51', 'b52', 'b53', 'b54', 'b55', 'b56', 'b57', 'b58', 'b59', 'b60', 'b61', 'b62', 'b63']

c = ['c00', 'c01', 'c02', 'c03', 'c04', 'c05', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15',
     'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31',
     'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47',
     'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63']

d = ['d00', 'd01', 'd02', 'd03', 'd04', 'd05', 'd06', 'd07', 'd08', 'd09', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15',
     'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26', 'd27', 'd28', 'd29', 'd30', 'd31',
     'd32', 'd33', 'd34', 'd35', 'd36', 'd37', 'd38', 'd39', 'd40', 'd41', 'd42', 'd43', 'd44', 'd45', 'd46', 'd47',
     'd48', 'd49', 'd50', 'd51', 'd52', 'd53', 'd54', 'd55', 'd56', 'd57', 'd58', 'd59', 'd60', 'd61', 'd62', 'd63']

# To this:
a_t = ['a00', 'a16', 'a32', 'a48', 'b00', 'b16', 'b32', 'b48', 'c00', 'c16', 'c32', 'c48', 'd00', 'd16', 'd32', 'd48',
       'a01', 'a17', 'a33', 'a49', 'b01', 'b17', 'b33', 'b49', 'c01', 'c17', 'c33', 'c49', 'd01', 'd17', 'd33', 'd49',
       'a02', 'a18', 'a34', 'a50', 'b02', 'b18', 'b34', 'b50', 'c02', 'c18', 'c34', 'c50', 'd02', 'd18', 'd34', 'd50',
       'a03', 'a19', 'a35', 'a51', 'b03', 'b19', 'b35', 'b51', 'c03', 'c19', 'c35', 'c51', 'd03', 'd19', 'd35', 'd51']

b_t = ['a04', 'a20', 'a36', 'a52', 'b04', 'b20', 'b36', 'b52', 'c04', 'c20', 'c36', 'c52', 'd04', 'd20', 'd36', 'd52',
       'a05', 'a21', 'a37', 'a53', 'b05', 'b21', 'b37', 'b53', 'c05', 'c21', 'c37', 'c53', 'd05', 'd21', 'd37', 'd53',
       'a06', 'a22', 'a38', 'a54', 'b06', 'b22', 'b38', 'b54', 'c06', 'c22', 'c38', 'c54', 'd06', 'd22', 'd38', 'd54',
       'a07', 'a23', 'a39', 'a55', 'b07', 'b23', 'b39', 'b55', 'c07', 'c23', 'c39', 'c55', 'd07', 'd23', 'd39', 'd55']

c_t = ['a08', 'a24', 'a40', 'a56', 'b08', 'b24', 'b40', 'b56', 'c08', 'c24', 'c40', 'c56', 'd08', 'd24', 'd40', 'd56',
       'a09', 'a25', 'a41', 'a57', 'b09', 'b25', 'b41', 'b57', 'c09', 'c25', 'c41', 'c57', 'd09', 'd25', 'd41', 'd57',
       'a10', 'a26', 'a42', 'a58', 'b10', 'b26', 'b42', 'b58', 'c10', 'c26', 'c42', 'c58', 'd10', 'd26', 'd42', 'd58',
       'a11', 'a27', 'a43', 'a59', 'b11', 'b27', 'b43', 'b59', 'c11', 'c27', 'c43', 'c59', 'd11', 'd27', 'd43', 'd59']

d_t = ['a12', 'a28', 'a44', 'a60', 'b12', 'b28', 'b44', 'b60', 'c12', 'c28', 'c44', 'c60', 'd12', 'd28', 'd44', 'd60',
       'a13', 'a29', 'a45', 'a61', 'b13', 'b29', 'b45', 'b61', 'c13', 'c29', 'c45', 'c61', 'd13', 'd29', 'd45', 'd61',
       'a14', 'a30', 'a46', 'a62', 'b14', 'b30', 'b46', 'b62', 'c14', 'c30', 'c46', 'c62', 'd14', 'd30', 'd46', 'd62',
       'a15', 'a31', 'a47', 'a63', 'b15', 'b31', 'b47', 'b63', 'c15', 'c31', 'c47', 'c63', 'd15', 'd31', 'd47', 'd63']
# fmt: on

# We start with the following transformation, applied to each 64-bit integer:
#
# x = ['x00', 'x01', 'x02', 'x03', 'x04', 'x05', 'x06', 'x07', 'x08', 'x09', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15',
#      'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31',
#      'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47',
#      'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63']
#
# x_t = ['x00', 'x16', 'x32', 'x48', 'x04', 'x20', 'x36', 'x52', 'x08', 'x24', 'x40', 'x56', 'x12', 'x28', 'x44', 'x60',
#        'x01', 'x17', 'x33', 'x49', 'x05', 'x21', 'x37', 'x53', 'x09', 'x25', 'x41', 'x57', 'x13', 'x29', 'x45', 'x61',
#        'x02', 'x18', 'x34', 'x50', 'x06', 'x22', 'x38', 'x54', 'x10', 'x26', 'x42', 'x58', 'x14', 'x30', 'x46', 'x62',
#        'x03', 'x19', 'x35', 'x51', 'x07', 'x23', 'x39', 'x55', 'x11', 'x27', 'x43', 'x59', 'x15', 'x31', 'x47', 'x63']


#  0 1 2 3 16 17 18 19 32 33 34 35 48 49 50 51 4 5 6 7 20 21 22 23 36 37 38 39 52 53 54 55 8 9 10 11 24 25 26 27 40 41 42 43 56 57 58 59 12 13 14 15 28 29 30 31 44 45 46 47 60 61 62 63
# |------------------- A ---------------------|------------------- B ---------------------|-------------------- C ----------------------|--------------------- D -----------------------|
# The source index for the bit permutation calculator can be generated by the following code:
# mapping = [x.index(x_t[i]) for i in range(64)]
# print(' '.join([str(i) for i in mapping]))


def print16x4(bits):
    for i in range(4):
        print(" ".join(str(bits[i * 16 + j]) for j in range(16)))
    print()


# Gather/shift opt: 9 cycles, 4 masks
# x = pext(x,  0x000f000f000f000f)
#   | (pext(x, 0x00f000f000f000f0) << 16)
#   | (pext(x, 0x0f000f000f000f00) << 32)
#   | (pext(x, 0xf000f000f000f000) << 48);


a = BitArray(a)
b = BitArray(b)
c = BitArray(c)
d = BitArray(d)

m0 = BitArray.from_int(0x000F000F000F000F, 64)
m1 = BitArray.from_int(0x00F000F000F000F0, 64)
m2 = BitArray.from_int(0x0F000F000F000F00, 64)
m3 = BitArray.from_int(0xF000F000F000F000, 64)

a = (
    compress(a, m0)
    | (compress(a, m1) << 16)
    | (compress(a, m2) << 32)
    | (compress(a, m3) << 48)
)
b = (
    compress(b, m0)
    | (compress(b, m1) << 16)
    | (compress(b, m2) << 32)
    | (compress(b, m3) << 48)
)
c = (
    compress(c, m0)
    | (compress(c, m1) << 16)
    | (compress(c, m2) << 32)
    | (compress(c, m3) << 48)
)
d = (
    compress(d, m0)
    | (compress(d, m1) << 16)
    | (compress(d, m2) << 32)
    | (compress(d, m3) << 48)
)

# m0 = BitArray.from_int(0x0000AAAA0000AAAA, 64)
# m1 = BitArray.from_int(0x00000000CCCCCCCC, 64)

# a = delta_swap(a, m0, 15)
# b = delta_swap(b, m0, 15)
# c = delta_swap(c, m0, 15)
# d = delta_swap(d, m0, 15)

# a = delta_swap(a, m1, 30)
# b = delta_swap(b, m1, 30)
# c = delta_swap(c, m1, 30)
# d = delta_swap(d, m1, 30)

# m = BitArray.from_int(0xF000_F000_F000_F000, 64)
# print16x4(m.bits)

# b = b.rotate_right(4)
# c = c.rotate_right(8)
# d = d.rotate_right(12)

# a, b = exchange(a, b, m.rotate_right(4))
# a, c = exchange(a, c, m.rotate_right(8))
# a, d = exchange(a, d, m.rotate_right(12))
# b, c = exchange(b, c, m.rotate_right(12))

print16x4(a.bits)
print16x4(b.bits)
print16x4(c.bits)
print16x4(d.bits)

ValueError: BitArray lengths must be equal.